In [ ]:
from fastai import *        # Quick access to most common functionality
from fastai.text import *   # Quick access to NLP functionality

# Text example

An example of creating a language model and then transfering to a classifier.

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ubuntu/fastai/examples/data/imdb_sample')

Open and view the independent and dependent variables:

In [ ]:
df = pd.read_csv(path/'texts.csv', header=None)
df.head()

,0,1,2
0,label,text,is_valid
1,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
2,positive,This is a extremely well-made film. The acting...,False
3,negative,Every once in a long while a movie will come a...,False
4,positive,Name just says it all. I watched this movie wi...,False


Create a `DataBunch` for each of the language model and the classifier:

In [ ]:
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=42)

We'll fine-tune the language model. [fast.ai](http://www.fast.ai/) has a pre-trained English model available that we can download, we jsut have to specify it like this:

In [ ]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103)
learn.unfreeze()
learn.fit(2, slice(1e-4,1e-2))

Total time: 00:44
epoch  train_loss  valid_loss  accuracy
1      4.891387    4.117615    0.254441  (00:22)
2      4.612432    4.050926    0.260979  (00:22)



In [ ]:
x,y = next(iter(learn.data.train_dl))

In [ ]:
z = learn.model[0](x)

In [ ]:
[t.size() for t in z[1]]

[torch.Size([67, 64, 1150]),
 torch.Size([67, 64, 1150]),
 torch.Size([67, 64, 400])]

Save our language model's encoder:

In [ ]:
learn.save_encoder('enc')

Fine tune it to create a classifier:

In [ ]:
learn = text_classifier_learner(data_clas)
learn.load_encoder('enc')
learn.fit(3, 1e-3)